# Split time column into time of day (4h buckets) (Early Morning/Late Morning/Early Noon/Late Noon/Early Evening/Late Evening) - Suman;

In [65]:
import pandas as pd

# Load the dataset into a DataFrame
df = pd.read_csv("/home/suman/code/soulseekqt/risky_predictive_policing/raw_data/chicago.csv")

#copy data into a different data frame
df_copy = df.copy()

# Remove leading and trailing spaces from each column name
df_copy.columns = df_copy.columns.str.strip()

# Apply strip() to each column where the data type is string (object)
df_copy = df_copy.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Display the first few rows
print(df_copy.head())

/tmp/ipykernel_5057/2881122101.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_copy = df_copy.applymap(lambda x: x.strip() if isinstance(x, str) else x)


      CASE#     DATE  OF OCCURRENCE                 BLOCK  IUCR  \
0  JH117298  01/16/2024 01:00:00 AM  038XX W DIVERSEY AVE  0810   
1  JG561057  12/31/2023 04:30:00 PM    004XX N WABASH AVE  0460   
2  JH117691  01/16/2024 06:50:00 PM       010XX W 99TH ST  143A   
3  JG522770  11/30/2023 07:28:00 AM   034XX W CHICAGO AVE  2024   
4  JG560426  12/31/2023 12:55:00 AM     0000X W GRAND AVE  143A   

  PRIMARY DESCRIPTION          SECONDARY DESCRIPTION LOCATION DESCRIPTION  \
0               THEFT                      OVER $500               STREET   
1             BATTERY                         SIMPLE               STREET   
2   WEAPONS VIOLATION  UNLAWFUL POSSESSION - HANDGUN               STREET   
3           NARCOTICS       POSSESS - HEROIN (WHITE)                ALLEY   
4   WEAPONS VIOLATION  UNLAWFUL POSSESSION - HANDGUN               STREET   

  ARREST DOMESTIC  BEAT  WARD FBI CD  X COORDINATE  Y COORDINATE   LATITUDE  \
0      N        N  2524    35     06     1150337.0     

In [66]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257077 entries, 0 to 257076
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CASE#                  257077 non-null  object 
 1   DATE  OF OCCURRENCE    257077 non-null  object 
 2   BLOCK                  257077 non-null  object 
 3   IUCR                   257077 non-null  object 
 4   PRIMARY DESCRIPTION    257077 non-null  object 
 5   SECONDARY DESCRIPTION  257077 non-null  object 
 6   LOCATION DESCRIPTION   256032 non-null  object 
 7   ARREST                 257077 non-null  object 
 8   DOMESTIC               257077 non-null  object 
 9   BEAT                   257077 non-null  int64  
 10  WARD                   257077 non-null  int64  
 11  FBI CD                 257077 non-null  object 
 12  X COORDINATE           257011 non-null  float64
 13  Y COORDINATE           257011 non-null  float64
 14  LATITUDE               257011 non-nu

In [67]:
print(df_copy.columns)

Index(['CASE#', 'DATE  OF OCCURRENCE', 'BLOCK', 'IUCR', 'PRIMARY DESCRIPTION',
       'SECONDARY DESCRIPTION', 'LOCATION DESCRIPTION', 'ARREST', 'DOMESTIC',
       'BEAT', 'WARD', 'FBI CD', 'X COORDINATE', 'Y COORDINATE', 'LATITUDE',
       'LONGITUDE', 'LOCATION'],
      dtype='object')


In [68]:
# Missing value in each column
df_copy.isnull().sum()

CASE#                       0
DATE  OF OCCURRENCE         0
BLOCK                       0
IUCR                        0
PRIMARY DESCRIPTION         0
SECONDARY DESCRIPTION       0
LOCATION DESCRIPTION     1045
ARREST                      0
DOMESTIC                    0
BEAT                        0
WARD                        0
FBI CD                      0
X COORDINATE               66
Y COORDINATE               66
LATITUDE                   66
LONGITUDE                  66
LOCATION                   66
dtype: int64

In [69]:
# Remove rows where any of the specified columns have missing data
df_copy = df_copy.dropna(subset=['X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION'])

# Verify the changes by checking the shape of the DataFrame
print(df_copy.shape)

(257011, 17)


In [70]:
# Missing value in each column
df_copy.isnull().sum()

CASE#                       0
DATE  OF OCCURRENCE         0
BLOCK                       0
IUCR                        0
PRIMARY DESCRIPTION         0
SECONDARY DESCRIPTION       0
LOCATION DESCRIPTION     1045
ARREST                      0
DOMESTIC                    0
BEAT                        0
WARD                        0
FBI CD                      0
X COORDINATE                0
Y COORDINATE                0
LATITUDE                    0
LONGITUDE                   0
LOCATION                    0
dtype: int64

In [71]:
# Convert 'DATE  OF OCCURRENCE' to datetime format
df_copy['DATE  OF OCCURRENCE'] = pd.to_datetime(df_copy['DATE  OF OCCURRENCE'], errors='coerce')

# Create a function to categorize time into buckets
def categorize_time(hour):
    if 0 <= hour < 6:
        return "Early Morning"
    elif 6 <= hour < 9:
        return "Late Morning"
    elif 9 <= hour < 12:
        return "Early Noon"
    elif 12 <= hour < 15:
        return "Late Noon"
    elif 15 <= hour < 18:
        return "Early Evening"
    else:
        return "Late Evening"

# Apply the time categorization to create 'TIME OF DAY' column
df_copy['TIME OF DAY'] = df_copy['DATE  OF OCCURRENCE'].dt.hour.apply(categorize_time)

In [72]:
df_copy.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,TIME OF DAY
0,JH117298,2024-01-16 01:00:00,038XX W DIVERSEY AVE,0810,THEFT,OVER $500,STREET,N,N,2524,35,06,1150337.0,1918345.0,41.931844,-87.722951,"(41.931843966, -87.722950868)",Early Morning
1,JG561057,2023-12-31 16:30:00,004XX N WABASH AVE,0460,BATTERY,SIMPLE,STREET,N,N,1834,42,08B,1176592.0,1902931.0,41.888994,-87.626935,"(41.888993854, -87.626934833)",Early Evening
2,JH117691,2024-01-16 18:50:00,010XX W 99TH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,2232,21,15,1170976.0,1839080.0,41.713905,-87.649425,"(41.713904887, -87.649424515)",Late Evening
3,JG522770,2023-11-30 07:28:00,034XX W CHICAGO AVE,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,Y,N,1121,27,18,1153117.0,1905117.0,41.895490,-87.713086,"(41.895490399, -87.713086271)",Late Morning
4,JG560426,2023-12-31 00:55:00,0000X W GRAND AVE,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,1831,42,15,1175975.0,1903895.0,41.891653,-87.629172,"(41.891653037, -87.62917162)",Early Morning


In [73]:
# Define the function to determine if the incident occurred on the weekend
def is_weekend(date):
    if (date.weekday() == 4 and date.hour >= 17):  # Friday 5 PM onwards
        return "Yes"
    elif date.weekday() in [5, 6]:  # Saturday or Sunday
        return "Yes"
    elif (date.weekday() == 0 and date.hour < 1):  # Monday before 1 AM
        return "Yes"
    else:
        return "No"

# Apply the function to the dataset
df_copy['WEEKEND'] = df_copy['DATE  OF OCCURRENCE'].apply(is_weekend)

# Verify the changes
print(df_copy[['DATE  OF OCCURRENCE', 'WEEKEND']].head())


  DATE  OF OCCURRENCE WEEKEND
0 2024-01-16 01:00:00      No
1 2023-12-31 16:30:00     Yes
2 2024-01-16 18:50:00      No
3 2023-11-30 07:28:00      No
4 2023-12-31 00:55:00     Yes


In [74]:
df_copy.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,TIME OF DAY,WEEKEND
0,JH117298,2024-01-16 01:00:00,038XX W DIVERSEY AVE,0810,THEFT,OVER $500,STREET,N,N,2524,35,06,1150337.0,1918345.0,41.931844,-87.722951,"(41.931843966, -87.722950868)",Early Morning,No
1,JG561057,2023-12-31 16:30:00,004XX N WABASH AVE,0460,BATTERY,SIMPLE,STREET,N,N,1834,42,08B,1176592.0,1902931.0,41.888994,-87.626935,"(41.888993854, -87.626934833)",Early Evening,Yes
2,JH117691,2024-01-16 18:50:00,010XX W 99TH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,2232,21,15,1170976.0,1839080.0,41.713905,-87.649425,"(41.713904887, -87.649424515)",Late Evening,No
3,JG522770,2023-11-30 07:28:00,034XX W CHICAGO AVE,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,Y,N,1121,27,18,1153117.0,1905117.0,41.895490,-87.713086,"(41.895490399, -87.713086271)",Late Morning,No
4,JG560426,2023-12-31 00:55:00,0000X W GRAND AVE,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,1831,42,15,1175975.0,1903895.0,41.891653,-87.629172,"(41.891653037, -87.62917162)",Early Morning,Yes


In [75]:
# Extract the month from 'DATE  OF OCCURRENCE' and create a new column 'MONTH'
df_copy['MONTH'] = df_copy['DATE  OF OCCURRENCE'].dt.month_name()

# Verify the changes
print(df_copy[['DATE  OF OCCURRENCE', 'MONTH']].head())

  DATE  OF OCCURRENCE     MONTH
0 2024-01-16 01:00:00   January
1 2023-12-31 16:30:00  December
2 2024-01-16 18:50:00   January
3 2023-11-30 07:28:00  November
4 2023-12-31 00:55:00  December


In [76]:
df_copy.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,TIME OF DAY,WEEKEND,MONTH
0,JH117298,2024-01-16 01:00:00,038XX W DIVERSEY AVE,0810,THEFT,OVER $500,STREET,N,N,2524,35,06,1150337.0,1918345.0,41.931844,-87.722951,"(41.931843966, -87.722950868)",Early Morning,No,January
1,JG561057,2023-12-31 16:30:00,004XX N WABASH AVE,0460,BATTERY,SIMPLE,STREET,N,N,1834,42,08B,1176592.0,1902931.0,41.888994,-87.626935,"(41.888993854, -87.626934833)",Early Evening,Yes,December
2,JH117691,2024-01-16 18:50:00,010XX W 99TH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,2232,21,15,1170976.0,1839080.0,41.713905,-87.649425,"(41.713904887, -87.649424515)",Late Evening,No,January
3,JG522770,2023-11-30 07:28:00,034XX W CHICAGO AVE,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,Y,N,1121,27,18,1153117.0,1905117.0,41.895490,-87.713086,"(41.895490399, -87.713086271)",Late Morning,No,November
4,JG560426,2023-12-31 00:55:00,0000X W GRAND AVE,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,1831,42,15,1175975.0,1903895.0,41.891653,-87.629172,"(41.891653037, -87.62917162)",Early Morning,Yes,December


In [77]:
# Define a mapping of specific offenses to broader categories
offense_mapping = {
    'THEFT': 'Property Crime',
    'MOTOR VEHICLE THEFT': 'Property Crime',
    'BURGLARY': 'Property Crime',
    'CRIMINAL DAMAGE': 'Property Crime',

    'BATTERY': 'Violent Crime',
    'ASSAULT': 'Violent Crime',
    'ROBBERY': 'Violent Crime',
    'HOMICIDE': 'Violent Crime',
    'CRIMINAL SEXUAL ASSAULT': 'Violent Crime',
    'STALKING': 'Violent Crime',
    'KIDNAPPING': 'Violent Crime',

    'NARCOTICS': 'Drug-Related Crime',
    'OTHER NARCOTIC VIOLATION': 'Drug-Related Crime',

    'PUBLIC PEACE VIOLATION': 'Public Order Crime',
    'PUBLIC INDECENCY': 'Public Order Crime',
    'LIQUOR LAW VIOLATION': 'Public Order Crime',

    'SEX OFFENSE': 'Sexual Offense',
    'CRIMINAL SEXUAL ASSAULT': 'Sexual Offense',

    'OTHER OFFENSE': 'Other Crime',
    'INTERFERENCE WITH PUBLIC OFFICER': 'Other Crime',
    'PROSTITUTION': 'Other Crime',
    'GAMBLING': 'Other Crime',
    'INTIMIDATION': 'Other Crime',
    'CONCEALED CARRY LICENSE VIOLATION': 'Other Crime',
    'OBSCENITY': 'Other Crime',
    'HUMAN TRAFFICKING': 'Other Crime',

    'NON-CRIMINAL': 'Non-Criminal'
}

# Apply the mapping to the 'PRIMARY DESCRIPTION' column
df_copy['OFFENSE GROUP'] = df_copy['PRIMARY DESCRIPTION'].map(offense_mapping).fillna('Other Crime')

# Verify the results
print(df_copy[['PRIMARY DESCRIPTION', 'OFFENSE GROUP']].head())
print(df_copy['OFFENSE GROUP'].value_counts())

  PRIMARY DESCRIPTION       OFFENSE GROUP
0               THEFT      Property Crime
1             BATTERY       Violent Crime
2   WEAPONS VIOLATION         Other Crime
3           NARCOTICS  Drug-Related Crime
4   WEAPONS VIOLATION         Other Crime
OFFENSE GROUP
Property Crime        119218
Violent Crime          79888
Other Crime            48158
Drug-Related Crime      5826
Sexual Offense          2739
Public Order Crime      1179
Non-Criminal               3
Name: count, dtype: int64


In [78]:
df_copy.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,...,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,TIME OF DAY,WEEKEND,MONTH,OFFENSE GROUP
0,JH117298,2024-01-16 01:00:00,038XX W DIVERSEY AVE,0810,THEFT,OVER $500,STREET,N,N,2524,...,06,1150337.0,1918345.0,41.931844,-87.722951,"(41.931843966, -87.722950868)",Early Morning,No,January,Property Crime
1,JG561057,2023-12-31 16:30:00,004XX N WABASH AVE,0460,BATTERY,SIMPLE,STREET,N,N,1834,...,08B,1176592.0,1902931.0,41.888994,-87.626935,"(41.888993854, -87.626934833)",Early Evening,Yes,December,Violent Crime
2,JH117691,2024-01-16 18:50:00,010XX W 99TH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,2232,...,15,1170976.0,1839080.0,41.713905,-87.649425,"(41.713904887, -87.649424515)",Late Evening,No,January,Other Crime
3,JG522770,2023-11-30 07:28:00,034XX W CHICAGO AVE,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,Y,N,1121,...,18,1153117.0,1905117.0,41.895490,-87.713086,"(41.895490399, -87.713086271)",Late Morning,No,November,Drug-Related Crime
4,JG560426,2023-12-31 00:55:00,0000X W GRAND AVE,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,Y,N,1831,...,15,1175975.0,1903895.0,41.891653,-87.629172,"(41.891653037, -87.62917162)",Early Morning,Yes,December,Other Crime
